In [1]:
import numpy as np
c = 299792458.0  # m/s

# Performance of FMCW radars

## General considerations

In [23]:
# Params
params = {
    'Tc': 1*10**-3,
    'chirp_delay': 2*10**-3,
    'N_frame': 8,
    'f0': 5.75*10**9,
    'B': 600*10**6,
    'adc_sampling': 1*10**6   
}
def update_params(params):
    params['lambda'] = c/params['f0']
    params['S'] = params['B']/params['Tc']  # Hz/s
    params['Tf'] = params['N_frame']*(params['Tc']+params['chirp_delay'])
    return params
params = update_params(params)

In [24]:
power = {
    'Pt': 10**((15)/10),
    'G': 10**((13)/10),
    'RCS': 1,
    'Pmin': 10**((-118)/10),
    'lambda': params['lambda']
}

In [25]:
usb = {
    'decimation': 3,
    'N_points_per_sweep': 1000,
    'dtype_byte': 2,
    'channels': 2
}

In [26]:
# Range metrics
def range_rx_snr(d):
    # Radar range equation
    return ((d['Pt']*d['G']**2*d['lambda']**2*d['RCS'])/(d['Pmin']*(4*np.pi)**3))**0.25
def range_adc_frequency(d):
    # The ADC cannot process a freq > sampling rate/2
    print(d['S'])
    print(d['adc_sampling'])
    return (c/(2*d['S']))*(d['adc_sampling'])
def range_chirp_delay(d):
    # Max range based on how long you wait between chirps
    return c*d['chirp_delay']/2
def range_resolution_if_freq(d):
    # Size of a bin based on the freq that can be separated by the FFT
    return c/(2*d['B'])
def range_resolution_if_phase(d):
    # Resolution based on phase measurements
    return 1

# Speed metrics
def velocity_chirp_duration(d):
    return d['lambda']/(4*d['Tc'])
def velocity_resolution_frame(d):
    return d['lambda']/(2*d['Tf'])

# USB metrics
def usb_tx_rate(d, p, divider=2**20):
    return d['N_points_per_sweep']*d['dtype_byte']*d['channels']/(
        d['decimation']*(p['Tc']+p['chirp_delay'])*divider)

In [27]:
def display_metrics(power, params):
    print("SPEED")
    print("Chirp duration max: {:.1f} m/s".format(velocity_chirp_duration(params)))
    print("Number of frames resolution: +/- {:.1f} m/s".format(velocity_resolution_frame(params)))
    print()
    print("RANGE")
    print("SNR max: \t{:.1f} m".format(range_rx_snr(power)))
    print("RT max: \t{:.1f} m".format(range_chirp_delay(params)))
    print("ADC max: \t{:.1f} m".format(range_adc_frequency(params)))
    print("Range resolution: +/- {:.1f} m".format(range_resolution_if_freq(params)))
    print()
    print("USB")
    usb_text = "USB transfer rate: {:.3f} Mb/s".format(usb_tx_rate(usb, params))
    if usb_tx_rate(usb, params) > 480*10**6/(8*2**20):
        usb_text = '[WARNING] ' + usb_text
    print(usb_text)

In [28]:
display_metrics(power, params)

SPEED
Chirp duration max: 13.0 m/s
Number of frames resolution: +/- 1.1 m/s

RANGE
SNR max: 	323.0 m
RT max: 	299792.5 m
600000000000.0
1000000
ADC max: 	249.8 m
Range resolution: +/- 0.2 m

USB
USB transfer rate: 0.424 Mb/s


## FMCW3

### Human size target

### Small target

In [263]:
params['Tc'] = 20*10**-6  # s
params['chirp_delay'] = Tc
params['adc_sampling'] = 2*10**6
R = 6.35*10**-3
R = 0.22*25.4*0.5*10**-3
power['RCS'] = np.pi*R**2
params = update_params(params)

In [264]:
display_metrics(power, params)

SPEED
Chirp duration max: 652.2 m/s
Number of frames resolution: +/- 81.5 m/s

RANGE
SNR max: 	22.7 m
RT max: 	3000.0 m
ADC max: 	5.0 m
Range resolution: +/- 0.2 m

USB
USB transfer rate: 31.789 Mb/s
